In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from gensim import models
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm
import os


In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
eat_info = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/강남구_카페_detail.csv")
eat_review = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/강남구_카페_review.csv")

In [46]:
eat_info

,Unnamed: 0,가게이름,가게 주소,음식 종류,가격 범위,리뷰개수,평점,lat,lng
0,1,새들러하우스,서울특별시 강남구 도산대로17길 10 B1,카페 / 디저트,만원 미만,104,4.4,37.519342,127.024908
1,2,테일러커피,서울특별시 강남구 강남대로160길 31 1F,카페 / 디저트,만원 미만,10,4.3,37.520120,127.021195
2,3,컨플릭트 스토어,서울특별시 강남구 논현로159길 46-5,카페 / 디저트,만원 미만,5,4.3,37.522314,127.024300
3,4,마일스톤커피,서울특별시 강남구 논현로159길 49,카페 / 디저트,만원 미만,69,4.2,37.521806,127.024323
4,5,우유니아이스크림,서울특별시 강남구 압구정로8길 5 1F,카페 / 디저트,만원 미만,146,4.1,37.522988,127.021339
...,...,...,...,...,...,...,...,...,...
1026,1027,리아라운지,서울특별시 강남구 도산대로 420 청담스퀘어B동 1F,카페 / 디저트,만원-2만원,3,NaN,NaN,NaN
1027,1028,Bar Tea Scent,서울특별시 강남구 선릉로162길 16 엘리자벳빌딩,카페 / 디저트,2만원-3만원,13,3.8,NaN,NaN
1028,1029,스케줄,서울특별시 강남구 선릉로152길 37 B1,카페 / 디저트,2만원-3만원,35,3.6,NaN,NaN
1029,1030,안서리베이킹랩,서울특별시 강남구 삼성로 755,카페 / 디저트,2만원-3만원,1,NaN,NaN,NaN


In [48]:
eat_review

,Unnamed: 0,가게이름,리뷰아이디,리뷰,맛평가
0,1,새들러하우스,젠히,인기 폭팔에 항상 솔드아웃이라 거의 방문이 불가능한 이메지너리 카페 인줄알았는데 평...,맛있다
1,2,새들러하우스,얌얌,새들러는 말모❤️ 크로플 맛집 많이 생겨도 여기가 최애! 사진은 케익 밖에 못찾겠지...,맛있다
2,3,새들러하우스,류슐랭가이드,<강민경 크로플 맛집!>맛있습니다. 다르긴 다르더라고요.바질을 좋아하는 저는 바질 ...,맛있다
3,4,새들러하우스,도도,받아서 바로 커피랑 먹으면 더 좋을텐데. 테이크아웃 전문점이라 조금 아쉬웠다. 그래...,맛있다
4,5,새들러하우스,소미,강민경 와플 크로플로 유명한 새들러하우스 (월요일 휴무) 저는 크로플 종류별로 들어...,맛있다
...,...,...,...,...,...
11299,11300,티하우스하다,둘기,처음 가본 티하우스 +_+ 티코스가 생소하면서도 궁금해서 밋업으로 다녀왔습니다! ...,맛있다
11300,11301,티하우스하다,맛되디,<나를 우리는 시간>강남구청역 인근에 위치한 티하우스. 다양한 차를 원하는 방식으로...,맛있다
11301,11302,티하우스하다,주아팍,"차와 빛만이 흐르는 곳, 티하우스 하다갤러리와 티하우스를 병행하는 공간으로 현재는 ...",맛있다
11302,11303,티하우스하다,토요미식회장,"갤러리와 티하우스입구에서부터 갤러리 느낌이 아주 강한, 그리고 창밖으로 도자기가 많...",맛있다


In [49]:
eat_info.columns

Index(['Unnamed: 0', '가게이름', '가게 주소', '음식 종류', '가격 범위', '리뷰개수', '평점', 'lat',
       'lng'],
      dtype='object')

In [50]:
eat_info.rename(columns={'가게이름': 'name'}, inplace =True)
eat_info.rename(columns={'리뷰': 'review'}, inplace =True)
eat_info.rename(columns={'평점': 'rating'}, inplace =True)
eat_info.rename(columns={'리뷰개수': 'rvw_cnt'}, inplace =True)
eat_info.rename(columns={'가게 주소': 'address'}, inplace =True)
eat_info.rename(columns={'음식 종류': 'category'}, inplace =True)
eat_info.rename(columns={'가격 범위': 'price'}, inplace =True)




eat_review.rename(columns={'가게이름': 'res_name'}, inplace =True)
eat_review.rename(columns={'리뷰아이디':  'user_name'}, inplace =True)
eat_review.rename(columns={'리뷰':  'review'}, inplace =True)
eat_review.rename(columns={'맛평가':  'rating'}, inplace =True)

eat_info.drop(['Unnamed: 0'],axis=1, inplace=True)
eat_review.drop(['Unnamed: 0'],axis=1, inplace=True)



In [51]:
eat_info.columns

Index(['name', 'address', 'category', 'price', 'rvw_cnt', 'rating', 'lat',
       'lng'],
      dtype='object')

In [52]:
eat_info = eat_info[eat_info['name'].notnull()]
eat_review = eat_review[eat_review['user_name'].notnull()]
eat_info.reset_index(drop=True,inplace=True)
eat_review.reset_index(drop=True,inplace=True)

In [53]:
#delete overlapped data
for id in eat_info.index:
    if list(eat_info['name']).count(eat_info.loc[id,'name'])>1:
        eat_info.drop(id,inplace=True)

In [54]:
#label encoding, Place_name -> p_id, User_name -> u_id
le = LabelEncoder()
le.fit(eat_info['name'])
eat_info['p_id'] = le.transform(eat_info['name'])
eat_review['p_id'] = le.transform(eat_review['res_name'])

user_le = LabelEncoder()
user_le.fit(eat_review['user_name'])
eat_review['u_id'] = user_le.transform(eat_review['user_name'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [55]:
# delete punctuation marks
def delete_punctuation_marks_review(review):
    review = review.replace('.', '. ');
    review = review.replace('!', '! ');
    review = review.replace('(', '( ');
    review = review.replace(')', ') ');
    review = review.replace('^', '');
    review = review.replace('*', '');
    review = review.replace('-', ' ');
    review = review.replace('\n', '. ');
    review = review.replace('\"', '. ');
    return review

In [56]:
eat_info.head()


,name,address,category,price,rvw_cnt,rating,lat,lng,p_id
0,새들러하우스,서울특별시 강남구 도산대로17길 10 B1,카페 / 디저트,만원 미만,104,4.4,37.519342,127.024908,350
1,테일러커피,서울특별시 강남구 강남대로160길 31 1F,카페 / 디저트,만원 미만,10,4.3,37.520120,127.021195,782
2,컨플릭트 스토어,서울특별시 강남구 논현로159길 46-5,카페 / 디저트,만원 미만,5,4.3,37.522314,127.024300,741
3,마일스톤커피,서울특별시 강남구 논현로159길 49,카페 / 디저트,만원 미만,69,4.2,37.521806,127.024323,225
5,마카롱쿠튀르,서울특별시 강남구 논현로175길 109,카페 / 디저트,만원 미만,28,4.1,37.522279,127.020596,228


In [57]:
eat_review.head()


,res_name,user_name,review,rating,p_id,u_id
0,새들러하우스,젠히,인기 폭팔에 항상 솔드아웃이라 거의 방문이 불가능한 이메지너리 카페 인줄알았는데 평...,맛있다,350,2182
1,새들러하우스,얌얌,새들러는 말모❤️ 크로플 맛집 많이 생겨도 여기가 최애! 사진은 케익 밖에 못찾겠지...,맛있다,350,1863
2,새들러하우스,류슐랭가이드,<강민경 크로플 맛집!>맛있습니다. 다르긴 다르더라고요.바질을 좋아하는 저는 바질 ...,맛있다,350,1296
3,새들러하우스,도도,받아서 바로 커피랑 먹으면 더 좋을텐데. 테이크아웃 전문점이라 조금 아쉬웠다. 그래...,맛있다,350,1185
4,새들러하우스,소미,강민경 와플 크로플로 유명한 새들러하우스 (월요일 휴무) 저는 크로플 종류별로 들어...,맛있다,350,1700


In [58]:
eat_info['reviews'] = ''


In [59]:
# from requests import Request
# from requests import Session


# def KOtoEN(review):
#     s=Session()

#     headers ={
#         "X-Naver-Client-Id": "NsHid2ZsKcP5QclyL66L",
#         "X-Naver-Client-Secret": "OPb4L85YOo",
#     }

#     payload ={
#         'source':'ko',
#         'target':'en',
#         'text': review,
#     }

#     url ="https://openapi.naver.com/v1/papago/n2mt"
#     # "https://openapi.naver.com/v1/language/translate"

#     req =Request('POST', url, data=payload, headers=headers)
#     prepped =req.prepare()
#     res =s.send(prepped)
#     print(res.json())
#     # result =res.json()['message']['result']['translatedText']
#     return result

In [60]:
review_1=eat_review['review'][1]
review_1

'새들러는 말모❤️ 크로플 맛집 많이 생겨도 여기가 최애! 사진은 케익 밖에 못찾겠지만 ㅠ 주문해서 먹었던 케익도 맛있고 크로플도 다 맛있었다. 종류별로 다 먹어봤고 다 맛있지만 여기만의 특별한 맛으로는 바질 크로플 추천! 호불호가 갈리는 메뉴지만 좋아하는 사람은 이것만 먹는다는 바질이 ㅋㅋㅋ 아주 살짝 매콤하게 도는 감칠맛이 자꾸 땡기게 한다 ㅋㅋㅋ 달달한거 하나랑 바질 하나랑 같이 먹으면 궁합이 딱 좋다 ㅎㅎ 특별한 날은 케익 주문하는 것도 추천..!! 크로플 눅눅하지 않고 생크림이랑 딸기랑 조합이 너무 좋아서 순삭 ㅠㅠ❤️'

In [61]:
# KOtoEN(review_1)

In [62]:
# ts = Translator()
# eat_review['review_eng'] = ''

# #translating review
# for id in tqdm(eat_review.index):
#     review = eat_review.loc[id,'review']
    
#     if type(review)==float:
#         continue
        
#     review_pre = delete_punctuation_marks_review(review)
    
#     try:
#         review_ts = KOtoEN(review_pre)
#     except:
#         review_ts = ''

#     print(review_ts)
    
#     eat_review.loc[id,'review_eng'] = review_ts

In [63]:
# ts = Translator()

# ts.translate(review_1, source='ko', target='en', verbose=False)

In [64]:
# ts = Translator()
# eat_review['review_eng'] = ''

# #translating review
# for id in tqdm(eat_review.index):
#     review = eat_review.loc[id,'review']
    
#     if type(review)==float:
#         continue
        
#     review_pre = delete_punctuation_marks_review(review)
    
#     try:
#         review_ts = ts.translate(review_pre, source='ko', target='en', verbose=False)
#     except:
#         review_ts = ''
    
#     eat_review.loc[id,'review_eng'] = review_ts

In [65]:
for id in eat_info['p_id']:
    review_all = ''
    
    for review in eat_review[eat_review['p_id']==id]['review']:
        review_all+=review +'. '
        
    eat_info.loc[eat_info['p_id']==id,'reviews'] = review_all

In [66]:
eat_info.head()


,name,address,category,price,rvw_cnt,rating,lat,lng,p_id,reviews
0,새들러하우스,서울특별시 강남구 도산대로17길 10 B1,카페 / 디저트,만원 미만,104,4.4,37.519342,127.024908,350,인기 폭팔에 항상 솔드아웃이라 거의 방문이 불가능한 이메지너리 카페 인줄알았는데 평...
1,테일러커피,서울특별시 강남구 강남대로160길 31 1F,카페 / 디저트,만원 미만,10,4.3,37.520120,127.021195,782,- 가로수길 카페 중 제일 자주 가게 되는 테일러커피입니다. 갈때마다 다른걸 먹어서...
2,컨플릭트 스토어,서울특별시 강남구 논현로159길 46-5,카페 / 디저트,만원 미만,5,4.3,37.522314,127.024300,741,☕️커피 애호가가 원하는 건 뭐든 찾을 수 있는 커피 편집샵 전세계의 다양한 커피...
3,마일스톤커피,서울특별시 강남구 논현로159길 49,카페 / 디저트,만원 미만,69,4.2,37.521806,127.024323,225,유명한 맛집에는 뭔가 이유가 있다.음식이 맛있거나분위기가 좋거나다시 기억날 정도로 ...
5,마카롱쿠튀르,서울특별시 강남구 논현로175길 109,카페 / 디저트,만원 미만,28,4.1,37.522279,127.020596,228,이번에 위치를 옮겨서 삐아프 초콜렛 전문 매장이랑 같이 운영하는거 같더라구요! 매장...


In [67]:
eat_info['price'].unique()

array(['만원 미만', '만원-2만원', '2만원-3만원', '4만원 이상', '3만원-4만원'], dtype=object)

In [68]:
eat_info['price'].replace('만원 미만',5000, inplace=True)
eat_info['price'].replace('만원-2만원',15000, inplace=True)
eat_info['price'].replace('2만원-3만원',25000, inplace=True)
eat_info['price'].replace('3만원-4만원',35000, inplace=True)
eat_info['price'].replace('4만원 이상',50000, inplace=True)

In [69]:
eat_review.head()


,res_name,user_name,review,rating,p_id,u_id
0,새들러하우스,젠히,인기 폭팔에 항상 솔드아웃이라 거의 방문이 불가능한 이메지너리 카페 인줄알았는데 평...,맛있다,350,2182
1,새들러하우스,얌얌,새들러는 말모❤️ 크로플 맛집 많이 생겨도 여기가 최애! 사진은 케익 밖에 못찾겠지...,맛있다,350,1863
2,새들러하우스,류슐랭가이드,<강민경 크로플 맛집!>맛있습니다. 다르긴 다르더라고요.바질을 좋아하는 저는 바질 ...,맛있다,350,1296
3,새들러하우스,도도,받아서 바로 커피랑 먹으면 더 좋을텐데. 테이크아웃 전문점이라 조금 아쉬웠다. 그래...,맛있다,350,1185
4,새들러하우스,소미,강민경 와플 크로플로 유명한 새들러하우스 (월요일 휴무) 저는 크로플 종류별로 들어...,맛있다,350,1700


In [70]:
eat_review['rating'].unique()

array(['맛있다', '별로', '괜찮다'], dtype=object)

In [71]:
eat_review['rating'].replace('맛있다',5, inplace=True)
eat_review['rating'].replace('괜찮다',3, inplace=True)
eat_review['rating'].replace('별로',0, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [72]:
eat_info

,name,address,category,price,rvw_cnt,rating,lat,lng,p_id,reviews
0,새들러하우스,서울특별시 강남구 도산대로17길 10 B1,카페 / 디저트,5000,104,4.4,37.519342,127.024908,350,인기 폭팔에 항상 솔드아웃이라 거의 방문이 불가능한 이메지너리 카페 인줄알았는데 평...
1,테일러커피,서울특별시 강남구 강남대로160길 31 1F,카페 / 디저트,5000,10,4.3,37.520120,127.021195,782,- 가로수길 카페 중 제일 자주 가게 되는 테일러커피입니다. 갈때마다 다른걸 먹어서...
2,컨플릭트 스토어,서울특별시 강남구 논현로159길 46-5,카페 / 디저트,5000,5,4.3,37.522314,127.024300,741,☕️커피 애호가가 원하는 건 뭐든 찾을 수 있는 커피 편집샵 전세계의 다양한 커피...
3,마일스톤커피,서울특별시 강남구 논현로159길 49,카페 / 디저트,5000,69,4.2,37.521806,127.024323,225,유명한 맛집에는 뭔가 이유가 있다.음식이 맛있거나분위기가 좋거나다시 기억날 정도로 ...
5,마카롱쿠튀르,서울특별시 강남구 논현로175길 109,카페 / 디저트,5000,28,4.1,37.522279,127.020596,228,이번에 위치를 옮겨서 삐아프 초콜렛 전문 매장이랑 같이 운영하는거 같더라구요! 매장...
...,...,...,...,...,...,...,...,...,...,...
1026,리아라운지,서울특별시 강남구 도산대로 420 청담스퀘어B동 1F,카페 / 디저트,15000,3,NaN,NaN,NaN,211,#일상속공간예술 #분위기짱 #맛은....청담스퀘어에 위치한 카페겸 바 리아라운지~ ...
1027,Bar Tea Scent,서울특별시 강남구 선릉로162길 16 엘리자벳빌딩,카페 / 디저트,25000,13,3.8,NaN,NaN,9,#압구정로데오 #바 아 여기 너무 좋았다 칵테일이 진짜 다 맛있었음. 제일 먼저 시...
1028,스케줄,서울특별시 강남구 선릉로152길 37 B1,카페 / 디저트,25000,35,3.6,NaN,NaN,383,4.7/5엄청난 빙수 맛집..!!복숭아 빙수 먹으러 갔는데 품절이라 샤인머스켓 먹음...
1029,안서리베이킹랩,서울특별시 강남구 삼성로 755,카페 / 디저트,25000,1,NaN,NaN,NaN,437,머랭으로 만드는 호주의 유명한 디저트인 ‘파블로바’ 케이크로 인스타에서 핫한 곳이에...


In [73]:
eat_review

,res_name,user_name,review,rating,p_id,u_id
0,새들러하우스,젠히,인기 폭팔에 항상 솔드아웃이라 거의 방문이 불가능한 이메지너리 카페 인줄알았는데 평...,5,350,2182
1,새들러하우스,얌얌,새들러는 말모❤️ 크로플 맛집 많이 생겨도 여기가 최애! 사진은 케익 밖에 못찾겠지...,5,350,1863
2,새들러하우스,류슐랭가이드,<강민경 크로플 맛집!>맛있습니다. 다르긴 다르더라고요.바질을 좋아하는 저는 바질 ...,5,350,1296
3,새들러하우스,도도,받아서 바로 커피랑 먹으면 더 좋을텐데. 테이크아웃 전문점이라 조금 아쉬웠다. 그래...,5,350,1185
4,새들러하우스,소미,강민경 와플 크로플로 유명한 새들러하우스 (월요일 휴무) 저는 크로플 종류별로 들어...,5,350,1700
...,...,...,...,...,...,...
11289,티하우스하다,둘기,처음 가본 티하우스 +_+ 티코스가 생소하면서도 궁금해서 밋업으로 다녀왔습니다! ...,5,801,1216
11290,티하우스하다,맛되디,<나를 우리는 시간>강남구청역 인근에 위치한 티하우스. 다양한 차를 원하는 방식으로...,5,801,1336
11291,티하우스하다,주아팍,"차와 빛만이 흐르는 곳, 티하우스 하다갤러리와 티하우스를 병행하는 공간으로 현재는 ...",5,801,2212
11292,티하우스하다,토요미식회장,"갤러리와 티하우스입구에서부터 갤러리 느낌이 아주 강한, 그리고 창밖으로 도자기가 많...",5,801,2403


In [74]:
eat_info.to_csv('/content/drive/MyDrive/Colab Notebooks/data/강남구_cafe_info.csv', index=False)
eat_review.to_csv('/content/drive/MyDrive/Colab Notebooks/data/강남구_cafe_review.csv', index=False)